In [1]:
!rm -rf /content/sample_data

In [ ]:
!wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/tandt_db.zip
!unzip tandt_db.zip
!rm -rf /content/tandt
!rm -rf /content/tandt_db.zip
!rm -rf /content/db/drjohnson

In [2]:
import shutil
import os

#Load the model

---



In [3]:
%cd /content
!git clone --recursive https://github.com/Isaaclabe/Hackaton-Real-Estate.git

/content
Cloning into 'Hackaton-Real-Estate'...
remote: Enumerating objects: 1634, done.
remote: Counting objects: 100% (1634/1634), done.
remote: Compressing objects: 100% (709/709), done.
remote: Total 1634 (delta 931), reused 1546 (delta 911), pack-reused 0 (from 0)
Receiving objects: 100% (1634/1634), 23.35 MiB | 12.42 MiB/s, done.
Resolving deltas: 100% (931/931), done.


In [4]:
!mkdir /content/Hackaton-Real-Estate/data

In [5]:
!wget https://demuc.de/colmap/vocab_tree_flickr100K_words32K.bin

--2025-05-21 20:11:59--  https://demuc.de/colmap/vocab_tree_flickr100K_words32K.bin
Resolving demuc.de (demuc.de)... 78.46.140.213, 2a01:4f8:d0a:528e::2
Connecting to demuc.de (demuc.de)|78.46.140.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15229678 (15M) [application/octet-stream]
Saving to: ‘vocab_tree_flickr100K_words32K.bin’

vocab_tree_flickr10 100%[===================>]  14.52M  8.06MB/s    in 1.8s    

2025-05-21 20:12:02 (8.06 MB/s) - ‘vocab_tree_flickr100K_words32K.bin’ saved [15229678/15229678]



# Config libraries

---



Setup FFmeg

In [ ]:
!apt-get update -qq
!apt-get install -y ffmpeg

Setup the Speedy-GS

In [7]:
!pip install -q plyfile

In [8]:
%cd /content/Hackaton-Real-Estate
!pip install -q /content/Hackaton-Real-Estate/submodules/diff-gaussian-rasterization

/content/Hackaton-Real-Estate
  Preparing metadata (setup.py) ... done


In [ ]:
%cd /content/Hackaton-Real-Estate/submodules/simple-knn
!python setup.py build_ext

In [10]:
os.makedirs("/content/Hackaton-Real-Estate/submodules/simple-knn", exist_ok=True)
source_path_3 = "/content/Hackaton-Real-Estate/submodules/simple-knn/build/lib.linux-x86_64-cpython-311/simple_knn/_C.cpython-311-x86_64-linux-gnu.so"
destination_folder_2 = "/content/Hackaton-Real-Estate/submodules/simple-knn/simple_knn"
shutil.move(source_path_3, destination_folder_2)

'/content/Hackaton-Real-Estate/submodules/simple-knn/simple_knn/_C.cpython-311-x86_64-linux-gnu.so'

In [11]:
%cd /content/Hackaton-Real-Estate

/content/Hackaton-Real-Estate


In [12]:
!mv /content/Hackaton-Real-Estate/submodules/simple-knn/simple_knn /content/Hackaton-Real-Estate

Setup Colmap

In [13]:
# Install Miniconda
!wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh > /dev/null 2>&1
!bash Miniconda3-latest-Linux-x86_64.sh -b -p /content/miniconda > /dev/null 2>&1

# Initialize conda for shell usage
!ln -s /content/miniconda/etc/profile.d/conda.sh /etc/profile.d/conda.sh > /dev/null 2>&1
!echo ". /content/miniconda/etc/profile.d/conda.sh" >> ~/.bashrc > /dev/null 2>&1

# Create a colmap environment
!/content/miniconda/bin/conda create -y -n colmap_env -c conda-forge colmap cudatoolkit > /dev/null 2>&1

In [14]:
# Make it executable
!chmod +x /content/Hackaton-Real-Estate/run_in_conda_env.sh

In [ ]:
# Install xvfb
!apt-get install -y xvfb

# FFmeg

---



In [16]:
!mkdir /content/extracted_frames/

In [30]:
!mkdir /content/extracted_frames/images

In [ ]:
!ffmpeg -i /content/IMG_3089.MOV -vf fps=12 -q:v 2 -threads 0 /content/extracted_frames/images/frame_%04d.jpg

In [29]:
!rm -rf  /content/extracted_frames/images

# Colmap

---



In [20]:
%cd /content/Hackaton-Real-Estate

/content/Hackaton-Real-Estate


In [37]:
!rm -rf /content/Hackaton-Real-Estate/data/sparse

In [ ]:
# Start a virtual display
!Xvfb :1 -screen 0 1024x768x24 > /dev/null 2>&1 &

# Run COLMAP
!DISPLAY=:1 ./run_in_conda_env.sh colmap automatic_reconstructor \
    --workspace_path /content/Hackaton-Real-Estate/data \
    --image_path /content/extracted_frames/images \
    --data_type video \
    --vocab_tree_path /content/vocab_tree_flickr100K_words32K.bin \
    --quality medium \
    --camera_model PINHOLE \
    --single_camera 1 \
    --sparse 1 \
    --dense 0 \
    --use_gpu 1 \
    --num_threads -1 #> /dev/null 2>&1

In [39]:
!mv /content/Hackaton-Real-Estate/data/sparse/project.ini /content/Hackaton-Real-Estate/data/sparse/0

mv: cannot stat '/content/Hackaton-Real-Estate/data/sparse/project.ini': No such file or directory


In [40]:
!rm -rf /content/Hackaton-Real-Estate/data/images
!cp -r /content/extracted_frames/images /content/Hackaton-Real-Estate/data/

In [41]:
!rm -rf /content/Hackaton-Real-Estate/data/database.db
!rm -rf /content/Hackaton-Real-Estate/data/database.db-shm
!rm -rf /content/Hackaton-Real-Estate/data/database.db-wal

In [ ]:
!DISPLAY=:1 ./run_in_conda_env.sh colmap model_converter \
    --input_path /content/Hackaton-Real-Estate/data/sparse/0 \
    --output_path /content/ \
    --output_type TXT

# Speedy-GS training

---



In [ ]:
!rm -rf /content/Hackaton-Real-Estate/result_ply

In [ ]:
!python train.py \
  -s /content/Hackaton-Real-Estate/data/ \
  -m /content/Hackaton-Real-Estate/result_ply \
  --iterations 20_000 \
  --test_iterations 30_000 \
  --visualize_iterations 30_000 #> /dev/null 2>&1

In [ ]:
file_path = '/content/Hackaton-Real-Estate/result_ply/point_cloud/iteration_20000/point_cloud.ply'
file_size_bytes = os.path.getsize(file_path)
file_size_gb = file_size_bytes / 1024**3

print(f"File size: {file_size_gb:.2f} GB")

# Viewer

In [ ]:
!git clone https://github.com/camenduru/splat
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb -O /content/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb


In [ ]:
import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['cloudflared', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(7860, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)

In [ ]:
%cd /content/Hackaton-Real-Estate/splat
!python -m http.server 7860